In [1]:
using TSSM
include("time_propagators.jl")

 *** OPENMP n_threads =           8


  likely near /home/alex/.julia/dev/TSSM/examples/time_propagators.jl:643
  likely near /home/alex/.julia/dev/TSSM/examples/time_propagators.jl:643
  likely near /home/alex/.julia/dev/TSSM/examples/time_propagators.jl:1014


step! (generic function with 6 methods)

# Test problem: nonlinear Schrödinger equation with time dependent potential

The soliton
$$\psi_\mathrm{ex}(x,t)=\frac{a\mathrm{e}^{\frac{1}{2}\mathrm{i}((a^2-b^2)t-bx)}}{\cosh(a(bt+x-c))}$$ 
is the exact solution of the cubic nonlinear Schrödinger
$$\mathrm{i}\frac{\partial}{\partial t}\psi(x,t)=-\frac{1}{2}\Delta\psi(x,t)+\kappa|\psi(x,t)|^2\psi(x,t),
\quad \psi(x,0) = \psi_\mathrm{ex}(x,0),$$
where $\kappa=-1$, and $a,b,c\in\mathbb{R}$ are arbitrarily chosen parameters. 

It follows that 
the exact solution of the *nonlinear* Schrödinger equation
$$\mathrm{i}\frac{\partial}{\partial t}\psi(x,t)=-\frac{1}{2}\Delta\psi(x,t)+V_\alpha(x,t)\psi(x,t)+\alpha\kappa|\psi(x,t)|^2\psi(x,t),\quad\psi(x,0) = \psi_\mathrm{ex}(x,0),$$
with *time dependent potential*
$$V_\alpha(x,t)= (1-\alpha)\kappa|\psi_\mathrm{ex}(x,t)|^2=\frac{(1-\alpha)\kappa a^2}{\cosh(a(bt+x-c))^2}$$
and, e.g., $\alpha=\frac{1}{2}$ is again given by $\psi_\mathrm{ex}(x,t)$.

We use this time-dependent problem to verify the correct implementation of various time propagation methods

In [2]:
# exact solution
function soliton(x, t)
    a = 2.0
    b = 1.0
    c = 0.0    
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [3]:
nx = 1024
xmin = -16
xmax = +16
V(x,t) = -0.5*abs(soliton(x,t)).^2
m = Schroedinger1D(nx, xmin, xmax, potential_t=V, cubic_coupling=-0.5)
psi = wave_function(m)
psi_ref = wave_function(m)

┌ Warning: `cfunction(f, r, a::Tuple)` is deprecated, use `cfunction(f, r, Tuple{a...})` instead.
│   caller = cfunction_check_return_type(::Function, ::Type, ::Tuple{DataType}) at TSSM.jl:492
└ @ TSSM /home/alex/.julia/dev/TSSM/src/TSSM.jl:492
┌ Warning: The function `cfunction` is now written as a macro `@cfunction`.
│   caller = cfunction(::Function, ::Type, ::Tuple{DataType}) at deprecated.jl:55
└ @ Base ./deprecated.jl:55
┌ Warning: `cfunction(f, r, a::Tuple)` is deprecated, use `cfunction(f, r, Tuple{a...})` instead.
│   caller = cfunction_check_return_type(::Function, ::Type, ::Tuple{DataType,DataType}) at TSSM.jl:492
└ @ TSSM /home/alex/.julia/dev/TSSM/src/TSSM.jl:492
┌ Warning: The function `cfunction` is now written as a macro `@cfunction`.
│   caller = cfunction(::Function, ::Type, ::Tuple{DataType,DataType}) at deprecated.jl:55
└ @ Base ./deprecated.jl:55


WfSchroedinger1D{Float64}(Ptr{Nothing} @0x00000000035787d0, Schroedinger1D{Float64}(Ptr{Nothing} @0x00000000036a46d0))

In [4]:
t0 = 0.0
tend = 1.0
set!(psi_ref, soliton, tend) # exact reference solution

┌ Warning: `cfunction(f, r, a::Tuple)` is deprecated, use `cfunction(f, r, Tuple{a...})` instead.
│   caller = set!(::WfSchroedinger1D{Float64}, ::Function, ::Float64) at tssm_common.jl:415
└ @ TSSM /home/alex/.julia/dev/TSSM/src/tssm_common.jl:415


# One-step methods

## Explicit Runge-Kutta
### Classical Runge-Kutta scheme of order 4

In [5]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:rk4)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01         NaN                    

┌ Warning: The start/next/done iteration protocol is deprecated. Implement `iterate(::EquidistantTimeStepper)`.
│   caller = ip:0x0
└ @ Core :-1


          16
  2   1.250e-01         NaN         NaN    NaN           32
  3   6.250e-02         NaN         NaN    NaN           64
  4   3.125e-02         NaN         NaN    NaN          128
  5   1.563e-02         NaN         NaN    NaN          256
  6   7.813e-03         NaN         NaN    NaN          512
  7   3.906e-03         NaN         NaN    NaN         1024
  8   1.953e-03         NaN         NaN    NaN         2048
  9   9.766e-04         NaN         NaN    NaN         4096
 10   4.883e-04   4.928e-13         NaN    NaN         8192


## Splitting Method
### 5-step composition of order 4 (Suzuki)

In [6]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
method = SplittingMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   5.413e-02                              20
  2   1.250e-01   8.118e-03   2.407e+00   2.74           40
  3   6.250e-02   5.281e-04   2.949e+01   3.94           80
  4   3.125e-02   1.003e-05   4.064e+03   5.72          160
  5   1.563e-02   2.574e-07   8.996e+02   5.28          320
  6   7.813e-03   1.548e-08   5.437e+00   4.06          640
  7   3.906e-03   9.598e-10   4.395e+00   4.01         1280
  8   1.953e-03   5.987e-11   4.187e+00   4.00         2560
  9   9.766e-04   3.792e-12   3.653e+00   3.98         5120
 10   4.883e-04   1.089e-12   9.914e-07   1.80        10240


### 3-step composition of order 4 (Yoshida)

In [7]:
set!(psi, soliton, t0)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
a, b = get_coeffs_composition(g)
method = SplittingMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   1.709e-01                              12
  2   1.250e-01   2.148e-02   1.083e+01   2.99           24
  3   6.250e-02   1.270e-03   1.038e+02   4.08           48
  4   3.125e-02   4.294e-05   9.731e+02   4.89           96
  5   1.563e-02   2.549e-06   5.830e+01   4.07          192
  6   7.813e-03   1.614e-07   3.945e+01   3.98          384
  7   3.906e-03   1.013e-08   4.224e+01   3.99          768
  8   1.953e-03   6.336e-10   4.317e+01   4.00         1536
  9   9.766e-04   3.962e-11   4.331e+01   4.00         3072
 10   4.883e-04   2.624e-12   2.439e+01   3.92         6144


## Splitting using RK4 for B step
### Suzuki 5-step order 4

In [8]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
method = SplittingRK4BMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   5.210e-02                              80
  2   1.250e-01   8.042e-03   2.187e+00   2.70          160
  3   6.250e-02   5.278e-04   2.845e+01   3.93          320
  4   3.125e-02   1.035e-05   3.565e+03   5.67          640
  5   1.563e-02   3.080e-07   4.439e+02   5.07         1280
  6   7.813e-03   1.881e-08   5.933e+00   4.03         2560
  7   3.906e-03   1.171e-09   5.196e+00   4.01         5120
  8   1.953e-03   7.319e-11   5.037e+00   4.00        10240
  9   9.766e-04   4.756e-12   3.544e+00   3.94        20480
 10   4.883e-04   1.160e-12   6.374e-06   2.04        40960


### Yoshida 3-step order 4

## Composition Method, B-part: implizit midpoint rule
### 5-step composition of order 4 (Suzuki), 3 fixed-point iterations

In [9]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
method = CompositionMethod(g,3)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.153e-02                              80
  2   1.250e-01   5.614e-03   2.273e+00   2.89          160
  3   6.250e-02   4.207e-04   1.335e+01   3.74          320
  4   3.125e-02   8.396e-06   2.650e+03   5.65          640
  5   1.563e-02   2.256e-07   5.992e+02   5.22         1280
  6   7.813e-03   1.433e-08   3.435e+00   3.98         2560
  7   3.906e-03   9.139e-10   3.343e+00   3.97         5120
  8   1.953e-03   5.785e-11   3.549e+00   3.98        10240
  9   9.766e-04   3.847e-12   2.274e+00   3.91        20480
 10   4.883e-04   1.141e-12   7.299e-07   1.75        40960


### 4 fixed-point iterations

In [10]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
method = CompositionMethod(g,4)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.309e-02                             100
  2   1.250e-01   6.348e-03   1.986e+00   2.76          200
  3   6.250e-02   4.656e-04   1.609e+01   3.77          400
  4   3.125e-02   1.545e-05   3.840e+02   4.91          800
  5   1.563e-02   8.905e-07   2.430e+01   4.12         1600
  6   7.813e-03   5.723e-08   1.264e+01   3.96         3200
  7   3.906e-03   3.627e-09   1.394e+01   3.98         6400
  8   1.953e-03   2.282e-10   1.478e+01   3.99        12800
  9   9.766e-04   1.407e-11   1.767e+01   4.02        25600
 10   4.883e-04   1.096e-12   1.709e+00   3.68        51200


### 3-step composition of order 4 (Yoshida), 3 fixed-point iterations

In [11]:
set!(psi, soliton, t0)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
method = CompositionMethod(g,3)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   9.449e-01                              48
  2   1.250e-01   2.432e-01   1.427e+01   1.96           96
  3   6.250e-02   1.409e-02   1.251e+03   4.11          192
  4   3.125e-02   4.193e-04   1.793e+04   5.07          384
  5   1.563e-02   1.015e-05   5.037e+04   5.37          768
  6   7.813e-03   3.930e-07   3.020e+03   4.69         1536
  7   3.906e-03   3.126e-08   1.951e+01   3.65         3072
  8   1.953e-03   2.312e-09   3.495e+01   3.76         6144
  9   9.766e-04   1.569e-10   7.556e+01   3.88        12288
 10   4.883e-04   1.050e-11   8.753e+01   3.90        24576


## Exponential Runge-Kutta 
### Krogstad scheme of order 4

In [12]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:krogstad)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   3.870e-02                              16
  2   1.250e-01   4.560e-03   2.787e+00   3.09           32
  3   6.250e-02   3.267e-04   1.240e+01   3.80           64
  4   3.125e-02   2.040e-05   2.148e+01   4.00          128
  5   1.563e-02   1.245e-06   2.410e+01   4.03          256
  6   7.813e-03   7.638e-08   2.338e+01   4.03          512
  7   3.906e-03   4.721e-09   2.217e+01   4.02         1024
  8   1.953e-03   2.933e-10   2.128e+01   4.01         2048
  9   9.766e-04   1.827e-11   2.075e+01   4.00         4096
 10   4.883e-04   1.158e-12   1.752e+01   3.98         8192


### Strehmel-Weiner scheme of order 4

In [13]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:strehmel_weiner)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   5.246e-02                              16
  2   1.250e-01   5.453e-03   4.854e+00   3.27           32
  3   6.250e-02   3.831e-04   1.573e+01   3.83           64
  4   3.125e-02   2.401e-05   2.482e+01   4.00          128
  5   1.563e-02   1.475e-06   2.748e+01   4.03          256
  6   7.813e-03   9.086e-08   2.695e+01   4.02          512
  7   3.906e-03   5.631e-09   2.589e+01   4.01         1024
  8   1.953e-03   3.503e-10   2.510e+01   4.01         2048
  9   9.766e-04   2.184e-11   2.457e+01   4.00         4096
 10   4.883e-04   1.376e-12   2.224e+01   3.99         8192


### Runge-Kutta-Lawson scheme of order 4

In [14]:
set!(psi, soliton, t0)
method = ExponentialRungeKutta(:lawson)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   1.144e-01                              16
  2   1.250e-01   1.482e-02   6.824e+00   2.95           32
  3   6.250e-02   9.790e-04   5.134e+01   3.92           64
  4   3.125e-02   4.565e-05   2.072e+02   4.42          128
  5   1.563e-02   2.844e-06   4.861e+01   4.00          256
  6   7.813e-03   1.796e-07   4.482e+01   3.98          512
  7   3.906e-03   1.127e-08   4.701e+01   3.99         1024
  8   1.953e-03   7.051e-10   4.787e+01   4.00         2048
  9   9.766e-04   4.405e-11   4.863e+01   4.00         4096
 10   4.883e-04   2.712e-12   5.622e+01   4.02         8192


# Multistep methods

In [15]:
type ExactSolution <: TimePropagationMethod
end
function step!(m::ExactSolution, psi::WaveFunction, 
         t0::Real, dt::Real, steps::Int, step::Int)
    set!(psi, soliton, t0+(step+1)*dt)
end    

┌ Warning: Deprecated syntax `type`.
│ Use `mutable struct` instead.
└ @ nothing none:1


step! (generic function with 7 methods)

## Exponential multistep
### exact evaluation of the integral; 4 steps; exact starting values; no correction step

In [16]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=0, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------


  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
┌ Warning: The start/next/done iteration protocol is deprecated. Implement `iterate(::EquidistantTimeStepper)`.
│   caller = iterate(::EquidistantTimeStepper) at essentials.jl:853
└ @ Base ./essentials.jl:853


  1   2.500e-01   4.888e-01                               4
  2   1.250e-01   6.979e-02   2.397e+01   2.81            8
  3   6.250e-02   4.556e-03   2.508e+02   3.94           16
  4   3.125e-02   2.948e-04   2.600e+02   3.95           32
  5   1.563e-02   1.849e-05   3.036e+02   3.99           64
  6   7.813e-03   1.153e-06   3.142e+02   4.00          128
  7   3.906e-03   7.193e-08   3.140e+02   4.00          256
  8   1.953e-03   4.490e-09   3.121e+02   4.00          512
  9   9.766e-04   2.804e-10   3.110e+02   4.00         1024
 10   4.883e-04   1.742e-11   3.262e+02   4.01         2048


### exact evaluation of the integral; 3 steps; exact starting values; one correction step

In [17]:
set!(psi, soliton, t0)
method = ExponentialMultistep(3, iters=1, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   1.382e-01                               6
  2   1.250e-01   2.286e-02   5.046e+00   2.60           14
  3   6.250e-02   2.362e-03   2.073e+01   3.27           30
  4   3.125e-02   1.645e-04   1.003e+02   3.84           62
  5   1.563e-02   1.031e-05   1.705e+02   4.00          126
  6   7.813e-03   6.355e-07   1.876e+02   4.02          254
  7   3.906e-03   3.930e-08   1.838e+02   4.02          510
  8   1.953e-03   2.441e-09   1.775e+02   4.01         1022
  9   9.766e-04   1.520e-10   1.735e+02   4.01         2046
 10   4.883e-04   9.385e-12   1.885e+02   4.02         4094


### Gauss quadrature of order 4 for the  integral; 3 steps; exact starting values; one correction step

In [18]:
set!(psi, soliton, t0)
gauss4 = QuadratureRule([(3-sqrt(3))/6, (3+sqrt(3))/6],[0.5,0.5])
method = ExponentialMultistep(3, iters=1, quadrature=gauss4, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   2.367e-01                               6
  2   1.250e-01   3.030e-02   1.445e+01   2.97           14


  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41


  3   6.250e-02   2.643e-03   4.569e+01   3.52           30
  4   3.125e-02   1.723e-04   1.462e+02   3.94           62
  5   1.563e-02   1.076e-05   1.818e+02   4.00          126
  6   7.813e-03   6.637e-07   1.949e+02   4.02          254
  7   3.906e-03   4.107e-08   1.911e+02   4.01          510
  8   1.953e-03   2.552e-09   1.850e+02   4.01         1022
  9   9.766e-04   1.588e-10   1.825e+02   4.01         2046
 10   4.883e-04   9.668e-12   2.265e+02   4.04         4094


### Exponential multistep version 2; 3 steps; exact starting values; one correction step

In [19]:
set!(psi, soliton, t0)
method = ExponentialMultistep(3, iters=1, version=2, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.537e-01                               6


  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41


  2   1.250e-01   1.058e-01   8.349e+00   2.10           14
  3   6.250e-02   1.086e-02   9.754e+01   3.28           30
  4   3.125e-02   2.048e-03   8.604e+00   2.41           62
  5   1.563e-02   2.102e-04   1.797e+02   3.28          126
  6   7.813e-03   1.475e-05   1.765e+03   3.83          254
  7   3.906e-03   9.275e-07   3.785e+03   3.99          510
  8   1.953e-03   5.729e-08   4.376e+03   4.02         1022
  9   9.766e-04   3.546e-09   4.301e+03   4.01         2046
 10   4.883e-04   2.202e-10   4.154e+03   4.01         4094


### Exponential multistep version 2; 3 steps; starting values by Suzuki composition method; one correction step

In [20]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
#starting_method = SplittingMethod(a,b)
starting_method = CompositionMethod(g,3)
method = ExponentialMultistep(3, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.319e-01                              46
  2   1.250e-01   1.053e-01   7.258e+00   2.04           54
 

  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41
  likely near /home/alex/.julia/packages/IJulia/0cLgR/src/kernel.jl:41


 3   6.250e-02   1.086e-02   9.625e+01   3.28           70
  4   3.125e-02   2.048e-03   8.570e+00   2.41          102
  5   1.563e-02   2.102e-04   1.798e+02   3.28          166
  6   7.813e-03   1.475e-05   1.765e+03   3.83          294
  7   3.906e-03   9.275e-07   3.785e+03   3.99          550
  8   1.953e-03   5.729e-08   4.376e+03   4.02         1062
  9   9.766e-04   3.546e-09   4.301e+03   4.01         2086
 10   4.883e-04   2.202e-10   4.153e+03   4.01         4134


### Exponential multistep version 2; 4 steps; starting values by Suzuki composition method; one correction step

In [21]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.768e-01                              65
  2   1.250e-01   9.169e-02   1.289e+01   2.38           73
  3   6.250e-02   1.203e-02   4.056e+01   2.93           89
  4   3.125e-02   1.326e-03   8.155e+01   3.18          121
  5   1.563e-02   5.244e-05   1.372e+04   4.66          185
  6   7.813e-03   1.204e-06   3.576e+05   5.44          313
  7   3.906e-03   3.721e-08   4.477e+04   5.02          569
  8   1.953e-03   1.340e-09   1.315e+04   4.80         1081
  9   9.766e-04   4.560e-11   2.188e+04   4.88         2105
 10   4.883e-04   1.609e-12   1.529e+04   4.83         4153


### Exponential multistep version 2; 5 steps; starting values by Suzuki composition method; one correction step

In [22]:
set!(psi, soliton, t0)
method = ExponentialMultistep(5, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.153e-02                              84
  2   1.250e-01   1.057e-01   6.411e-03  -1.35           92
  3   6.250e-02   1.422e-02   4.348e+01   2.89          108
  4   3.125e-02   9.562e-04   6.944e+02   3.89          140
  5   1.563e-02   2.730e-05   5.044e+04   5.13          204
  6   7.813e-03   7.407e-07   6.837e+04   5.20          332
  7   3.906e-03   1.416e-08   7.927e+05   5.71          588
  8   1.953e-03   2.227e-10   3.785e+06   5.99         1100
  9   9.766e-04   3.491e-12   3.894e+06   6.00         2124
 10   4.883e-04   3.389e-13   4.701e-02   3.36         4172


## Splitting using propagate_A and RK4 for B
### Suzuki 5-step order 4

In [23]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
method = SplittingRK4BMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   5.210e-02                              80
  2   1.250e-01   8.042e-03   2.187e+00   2.70          160
  3   6.250e-02   5.278e-04   2.845e+01   3.93          320
  4   3.125e-02   1.035e-05   3.565e+03   5.67          640
  5   1.563e-02   3.080e-07   4.439e+02   5.07         1280
  6   7.813e-03   1.881e-08   5.933e+00   4.03         2560
  7   3.906e-03   1.171e-09   5.196e+00   4.01         5120
  8   1.953e-03   7.319e-11   5.037e+00   4.00        10240
  9   9.766e-04   4.756e-12   3.544e+00   3.94        20480
 10   4.883e-04   1.160e-12   6.374e-06   2.04        40960


### Yoshida 3-step order 4

In [24]:
set!(psi, soliton, t0)
g = [1.351207191959657634, -1.702414383919315268, 1.351207191959657634] # Yoshida
a, b = get_coeffs_composition(g)
method = SplittingRK4BMethod(a,b)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.422e-01                              48
  2   1.250e-01   3.341e-02   7.747e+01   3.73           96
  3   6.250e-02   1.894e-03   1.834e+02   4.14          192
  4   3.125e-02   1.639e-04   3.379e+01   3.53          384
  5   1.563e-02   1.185e-05   8.295e+01   3.79          768
  6   7.813e-03   7.775e-07   1.485e+02   3.93         1536
  7   3.906e-03   4.948e-08   1.838e+02   3.97         3072
  8   1.953e-03   3.116e-09   2.000e+02   3.99         6144
  9   9.766e-04   1.955e-10   2.072e+02   3.99        12288
 10   4.883e-04   1.254e-11   1.662e+02   3.96        24576
